In [7]:
%load_ext autoreload
%autoreload 2
import numpy as np
import re
import time
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import textwrap
import scipy.stats as stats

from scipy import stats
import scipy
from functools import partial

import matplotlib.pylab as pylab
import seaborn as sns
sns.set_style('whitegrid', {'legend.frameon':True})
sns.set_palette(sns.color_palette("Set1", 12))
#sns.set_context("paper")
fontsize = 12
params = {'legend.fontsize': fontsize,
  'figure.figsize': (18, 15),
  'axes.labelsize': fontsize,
  'axes.titlesize':fontsize,
  'axes.edgecolor':"0.3",
  'xtick.labelsize':fontsize,
  'ytick.labelsize':fontsize,
  'legend.fontsize':10,
  'font.size':fontsize,
  'font.family':'serif'}
pylab.rcParams.update(params)
plt.rc('axes', labelsize=fontsize) 

#plt.style.use('ggplot')
%matplotlib inline

from tqdm.notebook import tqdm
import pandas as pd
from pathlib2 import Path
import os

In [1]:
import sys
sys.path.append('../')

from metric_store import save_metrics, save_metric, load_metrics, get_metric_names, load_metric, add_metric
from network_metrics import prepare_metric
from pipeline.pipeline import load_config

In [2]:
config_name = "pipeline.config"
config = load_config(config_name)

In [13]:
network_metric_names = ['LCC_0.9', 'GCC_0.9', 'degree_0.9', 'EVC_0.9', 'closeness_0.9',
                        'LCC_0.95', 'GCC_0.95', 'degree_0.95', 'EVC_0.95', 'closeness_0.95']
for network_metric_name in network_metric_names:
    metric_name = 'network_metrics/' + network_metric_name
    add_metric(config, metric_name, metric_name + '.npy')

In [3]:
from corr_network import load_data, get_available_mask
data = load_data(config)
available_mask = get_available_mask(data)

In [6]:
metric_names = ['local_grid_metrics_for_cyclones/network_metrics/LCC_w']
#metric_names = ['network_metrics/LCC_w']
for metric_name in metric_names:
    config.metrics_plot_options['metric_name'] = metric_name
    metric = load_metric(config, metric_name)
    metric = prepare_metric(metric_name, metric, available_mask)
    print(metric_name, metric.shape)

local_grid_metrics_for_cyclones/network_metrics/LCC_w ()


In [4]:
get_metric_names(config, prefix = '')

{'network_metrics/LCC': 'network_metrics/LCC.npy',
 'network_metrics/GCC': 'network_metrics/GCC.npy',
 'network_metrics/LCC_w': 'network_metrics/LCC_w.npy',
 'network_metrics/GCC_w': 'network_metrics/GCC_w.npy',
 'network_metrics/degree': 'network_metrics/degree.npy',
 'network_metrics/degree_w': 'network_metrics/degree_w.npy',
 'network_metrics/EVC': 'network_metrics/EVC.npy',
 'network_metrics/EVC_w': 'network_metrics/EVC_w.npy',
 'network_metrics/closeness': 'network_metrics/closeness.npy',
 'network_metrics/closeness_w': 'network_metrics/closeness_w.npy',
 'input_data/MSLP': WindowsPath('../../../data/ERA5/ERA5_MSLP_1982_2019_3h_0.75/resulting_cube_land_masked_and_preproc.npz'),
 'diff_metrics/network_metrics/LCC': 'diff_metrics/network_metrics/LCC.npy',
 'diff_metrics/network_metrics/GCC': 'diff_metrics/network_metrics/GCC.npy',
 'diff_metrics/network_metrics/LCC_w': 'diff_metrics/network_metrics/LCC_w.npy',
 'diff_metrics/network_metrics/GCC_w': 'diff_metrics/network_metrics/GCC_

In [7]:
grid_metrics = metric.item()

In [8]:
cur_cyclone = grid_metrics['1982_cyclone_1__1982-05-01_1982-05-05']
cur_cyclone = grid_metrics['1982_cyclone_2__1982-05-31_1982-06-05']
len(cur_cyclone['times'])

25

In [8]:
import os
os.chdir(r"D:\PC\UNN\Science\Climate\programs\climate-networks\src")
from plot_network_metrics.utils import get_times_lats_lots
from cyclones_info.cyclones_info import get_cyclones_info, get_cyclones
all_times, all_lats, all_lons = get_times_lats_lots(config)
cyclones_frame = get_cyclones_info(config)
cyclones_dict = get_cyclones(cyclones_frame, config.cyclone_metrics_options)

In [9]:
def plot_local_grid_metric(cur_cyclone, metric_name, image_path):
    cur_times = pd.to_datetime(cur_cyclone['times']).values
    cur_metric = np.array(cur_cyclone['metrics'])
    cur_means = np.array(cur_cyclone['means'])
    cur_stds = np.array(cur_cyclone['stds'])
    
    import matplotlib.dates as mdates
    fig = plt.figure(figsize = (7, 4), dpi = 200)
    ax = plt.gca()
    plt.plot(cur_times, cur_metric, label = 'Metric')
    plt.plot(cur_times, cur_means, label = 'Mean')
    plt.fill_between(cur_times, cur_means + cur_stds, cur_means - cur_stds, color = 'b', alpha = 0.15)

    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d %H:%M'))
    
    plt.title(metric_name)    
    fig.autofmt_xdate()
    plt.tight_layout()

    file_name = image_path / (metric_name + '.png')
    file_name.parent.mkdir(parents=True, exist_ok=True)
    
    plt.savefig(file_name)
    plt.close()

    
from plot_network_metrics.utils import create_dir, create_cyclone_metric_dir, create_cyclone_dir
cyclones_dir = create_dir(config)
metric_names = list(get_metric_names(config, prefix = 'local_grid_metrics_for_cyclones').keys())
for metric_name in tqdm(metric_names):
    config.metrics_plot_options['metric_name'] = metric_name
    metric = load_metric(config, metric_name)
    metric = prepare_metric(metric_name, metric, available_mask).item()
    print(metric_name, len(metric))
    for cid, (cyclone, (cyclone_name, cur_cyclone_metric)) in enumerate(zip(cyclones_dict, metric.items())):
        if np.sum(~np.isnan(cur_cyclone_metric['metrics'])) < 4:
            continue
        cyclone_metric_dir = create_cyclone_dir(config, cyclone, cyclones_dir)
        plot_local_grid_metric(cur_cyclone_metric, metric_name, image_path = cyclone_metric_dir)
    del metric

  0%|          | 0/28 [00:00<?, ?it/s]

local_grid_metrics_for_cyclones/network_metrics/LCC 353
local_grid_metrics_for_cyclones/network_metrics/LCC_w 353
local_grid_metrics_for_cyclones/network_metrics/degree 353
local_grid_metrics_for_cyclones/network_metrics/degree_w 353
local_grid_metrics_for_cyclones/network_metrics/EVC 353
local_grid_metrics_for_cyclones/network_metrics/EVC_w 353
local_grid_metrics_for_cyclones/network_metrics/closeness 353
local_grid_metrics_for_cyclones/network_metrics/closeness_w 353
local_grid_metrics_for_cyclones/input_data/MSLP 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/LCC 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/LCC_w 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/degree 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/degree_w 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/EVC 353
local_grid_metrics_for_cyclones/diff_metrics/network_metrics/EVC_w 353
local_grid_metrics_for_cyclones/diff_metrics/net

['local_grid_metrics_for_cyclones/network_metrics/LCC',
 'local_grid_metrics_for_cyclones/network_metrics/LCC_w',
 'local_grid_metrics_for_cyclones/network_metrics/degree',
 'local_grid_metrics_for_cyclones/network_metrics/degree_w',
 'local_grid_metrics_for_cyclones/network_metrics/EVC',
 'local_grid_metrics_for_cyclones/network_metrics/EVC_w',
 'local_grid_metrics_for_cyclones/network_metrics/closeness',
 'local_grid_metrics_for_cyclones/network_metrics/closeness_w',
 'local_grid_metrics_for_cyclones/input_data/MSLP',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/LCC',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/LCC_w',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/degree',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/degree_w',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/EVC',
 'local_grid_metrics_for_cyclones/diff_metrics/network_metrics/EVC_w',
 'local_grid_metrics_for_cyclones/diff_metrics/n

In [23]:
?get_metric_names

In [73]:
cyclone_metric_dir

WindowsPath('D:/Work/data/ERA5/ERA5_MSLP_1982_2019_3h_0.75/cyclones/1982_cyclone_1_1982-05-01_1982-05-05/network_metrics/LCC_w')

In [46]:
cur_times = pd.to_datetime(cur_cyclone['times'])
cur_times[0].hour

3

In [20]:
metric_names = list(get_metric_names(config).keys())
metric_names

['network_metrics/LCC',
 'network_metrics/GCC',
 'network_metrics/LCC_w',
 'network_metrics/GCC_w',
 'network_metrics/degree',
 'network_metrics/degree_w',
 'network_metrics/EVC',
 'network_metrics/EVC_w',
 'network_metrics/closeness',
 'network_metrics/closeness_w',
 'input_data/MSLP',
 'diff_metrics/network_metrics/LCC',
 'diff_metrics/network_metrics/GCC',
 'diff_metrics/network_metrics/LCC_w',
 'diff_metrics/network_metrics/GCC_w',
 'diff_metrics/network_metrics/degree',
 'diff_metrics/network_metrics/degree_w',
 'diff_metrics/network_metrics/EVC',
 'diff_metrics/network_metrics/EVC_w',
 'diff_metrics/network_metrics/closeness',
 'diff_metrics/network_metrics/closeness_w',
 'input_data/MSLP_preproc',
 'network_metrics/LCC_0.9',
 'network_metrics/GCC_0.9',
 'network_metrics/degree_0.9',
 'network_metrics/EVC_0.9',
 'network_metrics/closeness_0.9',
 'local_grid_metrics_for_cyclones/network_metrics/LCC',
 'local_grid_metrics_for_cyclones/network_metrics/LCC_w',
 'local_grid_metrics_fo

In [10]:
from cyclones_info.cyclones_info import get_only_known_data, get_lat_lon_for_cyclone, \
    create_cyclone_info_string, get_datetimes_for_cyclone_points, get_cyclone_for_special_number


def compute_mean_std(metric, cyclones, frame, all_times, all_lons, all_lats):
    import warnings
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", category=RuntimeWarning)
        means_array = np.nanmean(metric, axis=2)
        stds_array = np.nanstd(metric, axis=2, ddof=1)

    local_metric_means_stds = {}

    for cyclone in cyclones:
        cyclone_frame = get_cyclone_for_special_number(frame, cyclone['number'])
        cyclone_frame = get_only_known_data(cyclone_frame)
        c_lons, c_lats = get_lat_lon_for_cyclone(cyclone_frame)
        c_times = get_datetimes_for_cyclone_points(cyclone_frame)

        c_metric = []
        means = []
        stds = []
        for current_lon, current_lat, current_date_time in zip(c_lons, c_lats, c_times):
            ind_dt = np.where(all_times == current_date_time)[0][0]
            ind_lon = np.argmin(np.abs(all_lons - current_lon))
            ind_lat = np.argmin(np.abs(all_lats - current_lat))

            c_metric.append(metric[ind_lat, ind_lon, ind_dt])
            means.append(means_array[ind_lat, ind_lon])
            stds.append(stds_array[ind_lat, ind_lon])
        print(c_times)
        print(c_lats)
        print(c_lons)
        print(c_metric)
        sdf
        local_metric_means_stds[create_cyclone_info_string(cyclone)] = {'times': c_times,
                                                                        'metrics': c_metric,
                                                                        'means': means,
                                                                        'stds': stds}
    return local_metric_means_stds

In [28]:
local_metric_means_stds = compute_mean_std(metric, cyclones_dict, cyclones_frame,\
                                           all_times, all_lons, all_lats)


['1982.05.01 03:00:00', '1982.05.01 06:00:00', '1982.05.01 12:00:00', '1982.05.01 18:00:00', '1982.05.02 00:00:00', '1982.05.02 03:00:00', '1982.05.02 06:00:00', '1982.05.02 12:00:00', '1982.05.02 18:00:00', '1982.05.03 00:00:00', '1982.05.03 03:00:00', '1982.05.03 06:00:00', '1982.05.03 12:00:00', '1982.05.03 18:00:00', '1982.05.04 00:00:00', '1982.05.04 03:00:00', '1982.05.04 06:00:00', '1982.05.04 12:00:00', '1982.05.04 18:00:00', '1982.05.05 03:00:00']
[14.0, 14.5, 15.0, 15.5, 16.0, 16.5, 16.5, 16.5, 17.0, 17.0, 17.0, 17.5, 17.5, 17.5, 17.5, 17.5, 17.5, 17.5, 17.3, 17.2]
[82.5, 82.5, 82.0, 82.5, 83.0, 83.5, 84.0, 84.5, 85.0, 86.0, 86.5, 87.5, 88.5, 90.0, 91.0, 91.5, 92.0, 93.5, 94.5, 98.5]
[0.2817440927028656, 0.28558626770973206, 0.29499465227127075, 0.29902926087379456, 0.31387025117874146, 0.32767096161842346, 0.3433370888233185, 0.3656017780303955, 0.3561452329158783, 0.38406407833099365, 0.4089645743370056, 0.38848811388015747, 0.4161459803581238, 0.39044731855392456, 0.392942

NameError: name 'sdf' is not defined